In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import Trainer
from torch import optim

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy
import torch

import sys 
import os
sys.path.append(os.path.abspath('../networks'))
sys.path.append(os.path.abspath('../dev'))

from RNN import ModelConfig

preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

2025-06-04 20:19:01.004 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 3096.95it/s]


In [3]:
accuracy = Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

config = ModelConfig(
    input_size=3, # vast
    hidden_size=128,
    num_layers=2,
    output_size=20, # vast
    dropout=0.2,
)

In [4]:
from pathlib import Path

settings = TrainerSettings(
    epochs=10, 
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD],
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
    }
)
settings

2025-06-04 20:19:03.545 | INFO     | mltrainer.settings:check_path:60 - Created logdir /home/azureuser/machinelearning-melissa/notebooks/gestures


epochs: 10
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.1, 'patience': 10}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5}

In [5]:
from RNN import RecurrentNeuralNetwork

model = RecurrentNeuralNetwork(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

2025-06-04 18:52:05.731 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-185205
2025-06-04 18:52:06.855 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 69.88it/s]
2025-06-04 18:52:08.152 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.7248 test 2.4072 metric ['0.1359']
100%|██████████| 81/81 [00:01<00:00, 61.54it/s]
2025-06-04 18:52:09.553 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.5708 test 2.5407 metric ['0.1375']
2025-06-04 18:52:09.554 | INFO     | mltrainer.trainer:__call__:252 - best loss: 2.4072, current loss 2.5407.Counter 1/5.
100%|██████████| 81/81 [00:01<00:00, 60.95it/s]
2025-06-04 18:52:10.967 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.5318 test 2.4214 metric ['0.1344']
2025-06-04 18:52:10.969 | INFO     | mltrainer.trainer:__call__:252 - best loss: 2.4072, current loss 2.4214.Counte

In [6]:
from RNN import GRUWithAttention

model = GRUWithAttention(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

2025-06-04 18:52:19.637 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-185219
2025-06-04 18:52:19.638 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:03<00:00, 25.60it/s]
2025-06-04 18:52:22.978 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.0242 test 1.2945 metric ['0.5609']
100%|██████████| 81/81 [00:03<00:00, 26.05it/s]
2025-06-04 18:52:26.247 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.8294 test 0.4635 metric ['0.8797']
100%|██████████| 81/81 [00:03<00:00, 26.79it/s]
2025-06-04 18:52:29.444 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.2818 test 0.2098 metric ['0.9516']
100%|██████████| 81/81 [00:03<00:00, 26.30it/s]
2025-06-04 18:52:32.710 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.1366 test 0.1679 metric ['0.9641']
100%|██████████| 81/81 [00:03<00:00, 26.66it/s]
2025-06-04 18:52:35.92

In [8]:
from RNN import RecurrentNeuralNetworkWithAttention

model = RecurrentNeuralNetworkWithAttention(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

2025-06-04 18:53:13.020 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-185313
2025-06-04 18:53:13.021 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 67.34it/s]
2025-06-04 18:53:14.312 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.1782 test 1.6098 metric ['0.4766']
100%|██████████| 81/81 [00:01<00:00, 65.19it/s]
2025-06-04 18:53:15.645 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.3985 test 1.1000 metric ['0.6297']
100%|██████████| 81/81 [00:01<00:00, 63.38it/s]
2025-06-04 18:53:17.016 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.0790 test 0.8654 metric ['0.7734']
100%|██████████| 81/81 [00:01<00:00, 62.21it/s]
2025-06-04 18:53:18.411 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.8490 test 0.7339 metric ['0.7969']
100%|██████████| 81/81 [00:01<00:00, 65.82it/s]
2025-06-04 18:53:19.72

In [5]:
from RNN import RecurrentNeuralNetworkWithAttention

config = ModelConfig(
    input_size=3, # vast
    hidden_size=256,
    num_layers=5,
    output_size=20, # vast
    dropout=0.18,
)

model = RecurrentNeuralNetworkWithAttention(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

2025-06-04 20:19:12.815 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-201912
2025-06-04 20:19:13.732 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:06<00:00, 12.57it/s]
2025-06-04 20:19:20.945 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.3091 test 1.9223 metric ['0.2344']
100%|██████████| 81/81 [00:05<00:00, 14.37it/s]
2025-06-04 20:19:27.008 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.8908 test 1.7317 metric ['0.3531']
100%|██████████| 81/81 [00:05<00:00, 14.00it/s]
2025-06-04 20:19:33.221 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.6047 test 1.5660 metric ['0.4328']
100%|██████████| 81/81 [00:05<00:00, 14.37it/s]
2025-06-04 20:19:39.290 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.3007 test 1.1500 metric ['0.5766']
100%|██████████| 81/81 [00:05<00:00, 14.55it/s]
2025-06-04 20:19:45.29